# Extraction of the nearest pixels and gride points to the stations 

## 1. Importing python moduls 

In [ ]:
import numpy as np
import pandas as pd
import netCDF4 as nc
from netCDF4 import Dataset
import datetime as dt
import calendar as cld  
import xarray as xr
from os import listdir 

In [ ]:
# For parallelisation 
from dask.distributed import Client
client = Client(n_workers=16, threads_per_worker=2, memory_limit='32GB')
client

## 2. Set variables and import data

In [ ]:
threshold = '0'

path_lmdz_35km = '/home/hchaabani/Data/Snow/SCF/LMDZ-OR-Saloua/modified_data/monthly_values/monthly_values_threshold_'+threshold
path_esa = '/home/hchaabani/Data/Snow/SCFG/Satellite/ESA CCI/modified_data/monthly_values/monthly_values_threshold_'+threshold

snc_lmdz = xr.open_dataset( path_lmdz_35km + '/scf_lmdz_variables_monthly_values_threshold_'+threshold+'.nc')
esa_cci = xr.open_dataset( path_esa + '/ESA_CCI_GF_variables_monthly_values_threshold_'+threshold+'.nc') 

output_path_lmdz_35km = '/home/hchaabani/Data/Snow/SCFG/LMDZ-OR-Saloua/modified_data/extracted_mesh/extracted_mesh_threshold_'+threshold
output_path_esa_cci = '/home/hchaabani/Data/Snow/SCFG/Satellite/ESA CCI/modified_data/extracted_pixels/extracted_pixels_threshold_'+threshold

data = {'lmdz': snc_lmdz, 'esa': esa_cci}
output_paths = {'lmdz': output_path_lmdz_35km, 'esa': output_path_esa_cci}

In [ ]:
stations_labels = ['IFR','MID','TIC','MGN','TIZ','OUK']
stats_dic = {key:'data' for key in stations_labels} 

In [ ]:
lats = {'IFR':33.5,'MID':32.67,'TIC':31.53,'MGN':31.50,'TIZ':31.56,'OUK':31.20}
lons = {'IFR':-5.16,'MID':-4.74,'TIC':-6.30,'MGN':-6.45,'TIZ':-6.29,'OUK':-7.86}

In [ ]:
for key in stations_labels:    
    stats_dic[key] = {key:'data' for key in list(data.keys())}    

In [ ]:
stats_dic

In [ ]:
for stat in stations_labels:                          
    for key in list(data.keys()):     
        stats_dic[stat][key] = data[key].sel(lat=lats[stat],lon=lons[stat], method='nearest').load()

In [ ]:
stats_dic

In [ ]:
for stat in stations_labels:
    for key in list(data.keys()):
        stats_dic[stat][key].to_netcdf(output_paths[key] +'/monthly_values_'+stat+'_'+key+'.nc') 